In [ ]:
import requests
import pandas as pd
import xmltodict
import json
import tqdm

SEOUL_API_AUTH_KEY='****'
result=[]
# 장애인콜택시 이용 정보 공공데이터
# https://data.seoul.go.kr/dataList/OA-15558/A/1/datasetView.do
for dd in tqdm.tqdm_notebook(range(1,2)):
  url='http://openapi.seoul.go.kr:8088/{}/xml/disabledCalltaxi/1/5/202406{}'.format(SEOUL_API_AUTH_KEY,str(dd).zfill(2))
  response=requests.get(url)
  xml_data = response.content
  xml_dict=xmltodict.parse(xml_data)
  json_str=json.dumps(xml_dict,ensure_ascii=False, indent=2)
  json_data = json.loads(json_str)['openXMLSEOUL']['ROW']['list']['item']

  disabled_call_taxi_df=pd.DataFrame(json_data)
  disabled_call_taxi_df.columns=["차량고유번호","차량타입","예정일시","배차일시","승차일시","출발지구군","출발지상세","목적지구군","목적시상세"]
  disabled_call_taxi_df.to_excel('./disabled_call_taxi_202406.xlsx')

In [ ]:
import pandas as pd

disabled_pop=pd.read_excel('./disabled_pop_2018.xlsx',header=1)
disabled_pop.head()
disabled_pop_df =pd.DataFrame(disabled_pop)
disabled_pop_sum= disabled_pop_df.sum(numeric_only=True, axis=1)

In [ ]:
result = pd.concat([disabled_pop_df.loc[:,['유형(등급)']],disabled_pop_sum], axis=1)
result.columns=['si_gun_gu','total']
result.to_excel('./disabled_pop_sum.xlsx',index=False)

In [ ]:
#https://data.seoul.go.kr/dataList/18/S/2/datasetView.do
disabled_state = pd.read_excel('./disabled_state.xlsx',header=3,usecols=[1,2])
disabled_state.head()
disabled_state_df=pd.DataFrame(disabled_state)
disabled_state_df.columns=['si_gun_gu','total']
disabled_state_df = disabled_state_df.drop(index=0,axis=0)
disabled_state_df.to_excel('./disabled_state_2023_simple.xlsx',index=False)

In [ ]:
disabled_call_taxi = pd.read_excel('./disabled_call_taxi_202406.xlsx')
disabled_call_taxi_df = pd.DataFrame(disabled_call_taxi)
disabled_call_taxi_start_sgg_cnt=disabled_call_taxi_df['출발지구군'].value_counts()
disabled_call_taxi_start_sgg_cnt_df=pd.DataFrame(disabled_call_taxi_start_sgg_cnt)
disabled_call_taxi_start_sgg_cnt_df.columns =['total']

disabled_call_taxi_start_sgg_cnt_df.to_excel('./disabled_call_taxi_start_sgg_cnt.xlsx',index_label='si_gun_gu')

In [ ]:
disabled_call_taxi_arrival_sgg_cnt=disabled_call_taxi_df['목적지구군'].value_counts()
disabled_call_taxi_arrival_sgg_cnt_df=pd.DataFrame(disabled_call_taxi_arrival_sgg_cnt)
disabled_call_taxi_arrival_sgg_cnt_df.columns =['total']

disabled_call_taxi_arrival_sgg_cnt_df.to_excel('./disabled_call_taxi_arrival_sgg_cnt.xlsx',index_label='si_gun_gu')

In [ ]:
import pandas as pd
seoul_sgg_list =pd.read_excel('./seoul_sgg_list.xlsx')
seoul_sgg_list.columns=['code','si_gun_gu','lat','long']
disabled_pop =pd.read_excel('./disabled_state_2023_simple.xlsx')
disabled_pop.columns=['si_gun_gu','disabled_pop']
seoul_sgg_disabled_pop = pd.merge(seoul_sgg_list,disabled_pop,how='left',on='si_gun_gu')
seoul_sgg_disabled_pop.head()


In [ ]:
disabled_call_taxi_start_sgg_cnt = pd.read_excel('./disabled_call_taxi_start_sgg_cnt.xlsx')
disabled_call_taxi_start_sgg_cnt.columns=['si_gun_gu','start_point']
seoul_sgg_disabled_state = pd.merge(seoul_sgg_disabled_pop,disabled_call_taxi_start_sgg_cnt, how='left',on='si_gun_gu')
seoul_sgg_disabled_state.head()

In [ ]:
import pandas as pd

disabled_call_taxi_arrival_sgg_cnt = pd.read_excel('./disabled_call_taxi_arrival_sgg_cnt.xlsx')

disabled_call_taxi_arrival_sgg_cnt.columns=['si_gun_gu','arrival_point']
seoul_sgg_disabled_state_total = pd.merge(seoul_sgg_disabled_state,disabled_call_taxi_arrival_sgg_cnt, how='left',on='si_gun_gu')
seoul_sgg_disabled_state.head()

seoul_sgg_disabled_state_total = pd.read_excel('./seoul_sgg_disabled_state_total.xlsx')
for idx in seoul_sgg_disabled_state_total.index:
  start = seoul_sgg_disabled_state_total.loc[idx,'start_point']
  arrival = seoul_sgg_disabled_state_total.loc[idx,'arrival_point']
  seoul_sgg_disabled_state_total.loc[idx,'point_sum'] = start + arrival
seoul_sgg_disabled_state_total.head()
seoul_sgg_disabled_state_total.to_excel('./seoul_sgg_disabled_state_total.xlsx',index=False)

In [ ]:
import folium
from folium.features import DivIcon
import json
namsan=[37.55120584342893, 126.98346716392766]

seoul_sgg_geo = json.load(open('seoul_sgg.geojson',encoding='utf-8'))
seoul_disabled_taxi_use_bubble_map = folium.Map(
    location=namsan,
    tiles='CartoDB dark_matter',
    zoom_start=11
)
def style_map(feature):
  return {
    'opacity':0.7,
    'weight':1,
    'color':'white',
    'fillOpacity':0,
    'dashArray':'5, 5'
  }

folium.GeoJson(
    seoul_sgg_geo,
    style_function=style_map,
).add_to(seoul_disabled_taxi_use_bubble_map)

def add_sigungu_name_marker(font_color,map):
  for idx in seoul_sgg_disabled_state_total.index:
    lat=seoul_sgg_disabled_state_total.loc[idx,'lat']
    long=seoul_sgg_disabled_state_total.loc[idx,'long']
    name=seoul_sgg_disabled_state_total.loc[idx,'si_gun_gu']
    folium.map.Marker(
        # 위/경도 위치
        [lat, long],
        icon=DivIcon(
            # icon px 사이즈
            icon_size=(0, 0),
            # icon 좌 상단 위치 설정
            icon_anchor=(0, 0),
  
            # html 형식으로 text 추가
            # div 태그 안에 style 형식 추가
            html='<div\
                      style="\
                          font-size: 1.2rem;\
                          color: black;\
                          background-color:rgba(255, 255, 255, 0.4);\
                          width:45px;\
                          text-align:center;\
                          margin:0 20px 0 0;\
                      "><b>'
                 + '<span style="color:'+font_color+'; margin: 0px;">'+name+'</span>'
                 + '</b></div>',
        )).add_to(map)

add_sigungu_name_marker('white',seoul_disabled_taxi_use_bubble_map)
seoul_disabled_taxi_use_bubble_map

In [ ]:
taxi_use_total_mean =seoul_sgg_disabled_state_total['point_sum'].mean()
for idx in seoul_sgg_disabled_state_total.index:
  lat=seoul_sgg_disabled_state_total.loc[idx,'lat']
  long=seoul_sgg_disabled_state_total.loc[idx,'long']
  cnt=seoul_sgg_disabled_state_total.loc[idx,'point_sum']
  
  if cnt>taxi_use_total_mean:
    fill_color="#FF0000"
  else:
    fill_color="#00A3A5"

  folium.CircleMarker(
      location=[lat, long],
      color='#FFFF00',
      fill_color=fill_color,
      fill_opacity=0.7,
      weight=1.5,
      radius=cnt/50
        ).add_to(seoul_disabled_taxi_use_bubble_map)

# seoul_disabled_taxi_use_bubble_map
seoul_disabled_taxi_use_bubble_map.save(outfile='./seoul_disabled_taxi_use_bubble_map_with_point_sum.html')

In [ ]:
taxi_use_total_mean =seoul_sgg_disabled_state_total['start_point'].mean()
for idx in seoul_sgg_disabled_state_total.index:
  lat=seoul_sgg_disabled_state_total.loc[idx,'lat']
  long=seoul_sgg_disabled_state_total.loc[idx,'long']
  cnt=seoul_sgg_disabled_state_total.loc[idx,'start_point']

  if cnt>taxi_use_total_mean:
    fill_color="#FF0000"
  else:
    fill_color="#00A3A5"

  folium.CircleMarker(
      location=[lat, long],
      color='#FFFF00',
      fill_color=fill_color,
      fill_opacity=0.7,
      weight=1.5,
      radius=cnt/30
  ).add_to(seoul_disabled_taxi_use_bubble_map)

# seoul_disabled_taxi_use_bubble_map
seoul_disabled_taxi_use_bubble_map.save(outfile='./seoul_disabled_taxi_use_bubble_map_with_start_point.html')

In [ ]:
taxi_use_total_mean =seoul_sgg_disabled_state_total['arrival_point'].mean()
for idx in seoul_sgg_disabled_state_total.index:
  lat=seoul_sgg_disabled_state_total.loc[idx,'lat']
  long=seoul_sgg_disabled_state_total.loc[idx,'long']
  cnt=seoul_sgg_disabled_state_total.loc[idx,'arrival_point']

  if cnt>taxi_use_total_mean:
    fill_color="#FF0000"
  else:
    fill_color="#00A3A5"

  folium.CircleMarker(
      location=[lat, long],
      color='#FFFF00',
      fill_color=fill_color,
      fill_opacity=0.7,
      weight=1.5,
      radius=cnt/30
  ).add_to(seoul_disabled_taxi_use_bubble_map)

# seoul_disabled_taxi_use_bubble_map
seoul_disabled_taxi_use_bubble_map.save(outfile='./seoul_disabled_taxi_use_bubble_map_with_arrival_point.html')

In [ ]:
starbucks_choropleth_map=folium.Map(
    location=namsan,
    zoom_start=11
)

folium.Choropleth(
    geo_data=seoul_sgg_geo,
    data=seoul_sgg_disabled_state_total,
    columns=['si_gun_gu','disabled_pop'],
    fill_color='YlGnBu',
    fill_opacity=0.6,
    line_opacity=1,
    key_on='properties.SIG_KOR_NM',
    legend_name="시군구별 장애인구수"
).add_to(starbucks_choropleth_map)

add_sigungu_name_marker('black',starbucks_choropleth_map)

starbucks_choropleth_map.save('./seoul_sgg_disabled_state_population.html')

In [ ]:
seoul_sgg_disabled_state_total['uses_per_100'] = seoul_sgg_disabled_state_total['point_sum']/(seoul_sgg_disabled_state_total['disabled_pop']/100)
top_use_sigungu = seoul_sgg_disabled_state_total['uses_per_100'].quantile(0.9)

for idx in seoul_sgg_disabled_state_total.index:
  lat=seoul_sgg_disabled_state_total.loc[idx,'lat']
  long=seoul_sgg_disabled_state_total.loc[idx,'long']
  cnt=seoul_sgg_disabled_state_total.loc[idx,'uses_per_100']
  
  if cnt>top_use_sigungu:
    fill_color="#fcba03"
  else:
    fill_color="#03f0fc"

  folium.CircleMarker(
      location=[lat, long],
      color='#FFFF00',
      fill_color=fill_color,
      fill_opacity=0.7,
      weight=1.5,
      radius=cnt/30
  ).add_to(seoul_disabled_taxi_use_bubble_map)

# seoul_disabled_taxi_use_bubble_map
seoul_disabled_taxi_use_bubble_map.save(outfile='./seoul_disabled_taxi_use_bubble_map_with_population_as_users.html')